In [582]:
import asyncio
import aioschedule
from telebot.async_telebot import AsyncTeleBot
from telebot import types
from asyncio import sleep
import datetime
import gspread
import random

In [583]:
(datetime.datetime.today() - datetime.timedelta(days=1)).date()

datetime.date(2023, 4, 30)

In [584]:
class Game:
    def __init__(self, title, url, is_premium, activity, age, num_of_parts, time, smile, aim):
        self.title = title
        self.url = url
        self.is_premium = is_premium
        self.activity = activity
        self.age = age
        self.num_of_participants = num_of_parts
        self.time = time
        self.smile = smile
        self.aim = aim
    def get_description(self):
        ages = ''
        if len(self.age) == 1:
            ages += self.age[0]
        elif len(self.age) == 2:
            ages += f'{self.age[0]}, {self.age[1]}'
        else:
            ages = 'Любой возраст'
        mess = f'[{self.title}]({self.url}) {self.smile}\n{self.activity} | {self.num_of_participants} | {ages}\n\n'
        return mess
all_games = []
games_no_premium = []

In [585]:
class Song:
    def __init__(self, title, url, smile, is_premium):
        self.title = title
        self.url = url
        self.smile = smile
        self.is_premium = is_premium
    def get_description(self):
        mess = f'[{self.title}]({self.url}) {self.smile}\n'
        return mess
all_songs = []
songs_no_premium = []

In [586]:
class User:
    def __init__(self, name, age, tgid):
        self.name = name
        self.age = age
        self.pid = tgid
        self.position = 'main_menu'
        self.is_premium = False
        self.premium_ends = (datetime.datetime.today() - datetime.timedelta(days=1))
    def check_premium(self):
        if self.is_premium and self.premium_ends < datetime.datetime.today():
            self.is_premium = False
    def getinfo(self):
        prev = ''
        if not self.is_premium:
            prev = 'Бесплатный'
        else:
            date_end = self.premium_ends.strftime("%d.%m")
            prev = f'Premium\nДата окончания: {date_end}'
        message = f"Имя: {self.name}\nВозраст: {self.age}\nPersonalID: {self.pid}\nТариф: {prev}"
        return message
    def avaliable_games(self):
        if not self.is_premium:
            return games_no_premium
        return all_games
    def avaliable_songs(self):
        if not self.is_premium:
            return songs_no_premium
        return all_songs
registered_users = dict()
current_register = dict()
registration_answers = dict()

In [587]:
#get sheets from google sheet
gc = gspread.service_account('vozhatnik_key.json')
db = gc.open('Database_Games_Vozhatnik')

games_db = db.worksheet('Все игры')
songs_db = db.worksheet('Все песни')
users_db = db.worksheet('Пользователи')

In [588]:
def parse_games():
    global games_db
    global all_games
    global games_no_premium
    all_games = []
    games_no_premium = []
    names = games_db.col_values(1)
    urls = games_db.col_values(2)
    is_prems = games_db.col_values(3)
    activities = games_db.col_values(4)
    ages = [i.split(', ') for i in games_db.col_values(5)]
    num_parts = games_db.col_values(6)
    times = games_db.col_values(7)
    smiles = games_db.col_values(8)
    aims = games_db.col_values(9)
    for i in range(len(names)):
        game_i = Game(names[i], urls[i], int(is_prems[i]), activities[i], ages[i], num_parts[i], times[i], smiles[i], aims[i])
        all_games.append(game_i)
        if game_i.is_premium == 0:
            games_no_premium.append(game_i)
parse_games()
print(len(all_games), len(games_no_premium))

82 31


In [589]:
def parse_songs():
    global songs_db
    global all_songs
    global songs_no_premium
    all_songs = []
    songs_no_premium = []
    names = songs_db.col_values(1)
    urls = songs_db.col_values(2)
    smiles = songs_db.col_values(3)
    is_prems = songs_db.col_values(4)
    for i in range(len(names)):
        song_i = Song(names[i], urls[i], smiles[i], int(is_prems[i]))
        all_songs.append(song_i)
        if song_i.is_premium == 0:
            songs_no_premium.append(song_i)
parse_songs()
print(len(all_songs), len(songs_no_premium))

22 20


In [590]:
q1 = '1. Как вас зовут?'
q2 = '2. Сколько вам лет?'
q3 = '3. Как вы узнали о боте?'

registration_question = [q1, q2, q3]

In [591]:
back_but = types.KeyboardButton("Назад")

main_menu_markup =  types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
my_account_but = types.KeyboardButton("Найти песни")
find_games_but = types.KeyboardButton("Найти игры")
find_songs_but = types.KeyboardButton("Мой аккаунт")
main_menu_markup.add(my_account_but, find_games_but, find_songs_but)

my_account_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
edit_data_account_but = types.KeyboardButton("Изменить данные аккаунта")
my_account_markup.add(back_but, edit_data_account_but)

change_account_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
name_but = types.KeyboardButton("Имя")
age_but = types.KeyboardButton("Возраст")
change_account_markup.add(name_but, age_but, back_but)

find_songs_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
random_song_but = types.KeyboardButton("Рандомная песня")
all_songs_but = types.KeyboardButton("Все песни")
find_songs_markup.add(random_song_but, all_songs_but, back_but)

find_games_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
random_game_but = types.KeyboardButton("Рандомная игра")
random_game_category_but = types.KeyboardButton("Рандомная игра определенной категории")
game_filter_but = types.KeyboardButton("Найти игры по фильтрам")
find_games_markup.add(random_game_but, random_game_category_but, game_filter_but, back_but)

aims_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
a1 = types.KeyboardButton("Знакомство")
a2 = types.KeyboardButton("Командообразование")
a3 = types.KeyboardButton("Снятие напряжения")
a4 = types.KeyboardButton("Игры-розыгрыши")
a5 = types.KeyboardButton("Игры с залом")
a6 = types.KeyboardButton("В автобусе")
a7 = types.KeyboardButton("Тактилка")
no_matter_but = types.KeyboardButton("Неважно")
aims_markup.add(a1, a2, a3, a4, a5, a6, a7, no_matter_but)

activity_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
act1 = types.KeyboardButton("Спокойная")
act2 = types.KeyboardButton("Активная")
act3 = types.KeyboardButton("Подвижная")
activity_markup.add(act1, act2, act3, no_matter_but)

age_category_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
age1 = types.KeyboardButton("Младший")
age2 = types.KeyboardButton("Средний")
age3 = types.KeyboardButton("Старший")
age_category_markup.add(age1, age2, age3, no_matter_but)

num_of_parts_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
num1 = types.KeyboardButton("2")
num2 = types.KeyboardButton("10-30")
num3 = types.KeyboardButton("30-50")
num4 = types.KeyboardButton("♾")
num_of_parts_markup.add(num1, num2, num3, num4, no_matter_but)

time_markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2, one_time_keyboard=True)
t1 = types.KeyboardButton("1-5")
t2 = types.KeyboardButton("5-10")
t3 = types.KeyboardButton("10-20")
time_markup.add(t1, t2, t3, no_matter_but)

questions_filter = ['Выберите тип активности', 'Выберите возрастную категорию ребят', 'Выберите количество участников', 'Выберите длительность игры']
markups_filter = [activity_markup, age_category_markup, num_of_parts_markup, time_markup]

In [592]:
TOKEN = '5706856021:AAHBLAFnHVrw-Ms33FjejW9JQH-HycjayeE'

bot = AsyncTeleBot(TOKEN)

@bot.message_handler(commands=['start'])
async def hello_message(message):
    user = message.chat.id
    await sleep(0.5)
    if user in registered_users.keys():
        await bot.send_message(user, 'Вы уже зарегистрированы. Переходим в главное меню', reply_markup = main_menu_markup)
        registered_users[user].position = 'main_menu'
    elif user in current_register:
        await bot.send_message(user, 'Пожалуйста, продолжите регистрацию')
        await sleep(0.5)
        await bot.send_message(user, registration_question[current_register[user]])
    else:
        mess1 = 'Привет! Добро пожаловать в главный помошник вожатого - "Галстук". Данный бот - огромный сборник стихов, песен, заметок, контактов и других незаменимых в жизни вожатого вещей'
        mess2 = 'Перед тем как начать, пожалуйста, пройди небольшую регистрацию. Она займет не больше минуты. Честно!'
        mess3 = 'Введи команду /register, чтобы начать регистрацию'
        await bot.send_message(user, mess1)
        await sleep(0.5)
        await bot.send_message(user, mess2)
        await sleep(0.5)
        await bot.send_message(user, mess3)

        
@bot.message_handler(commands=['register'])
async def hello_message(message):
    user = message.chat.id
    await sleep(0.5)
    if user in registered_users.keys():
        await bot.send_message(user, 'Вы уже зарегистрированы. Перехожу в главное меню', reply_markup = main_menu_markup)
        registered_users[user].position = 'main_menu_markup'
    elif user not in current_register.keys():
        current_register[user] = 0
        registration_answers[user] = []
        await bot.send_message(user, 'Отлично! Начинаем регистрацию!')
        await sleep(0.5)
        await bot.send_message(user, registration_question[0])
    else:
        await bot.send_message(user, 'Пожалуйста, продолжите регистрацию')
        await sleep(0.5)
        await bot.send_message(user, registration_question[current_register[user]])

@bot.message_handler(content_types=['text'])
async def message_reply(message):
    await sleep(0.5)
    user = message.chat.id
    if user in registered_users.keys():
        user = registered_users[user]
        if user.position == 'main_menu':
            if message.text == 'Мой аккаунт':
                user.position = 'my_account'
                await bot.send_message(user.pid, user.getinfo(), reply_markup = my_account_markup)
            elif message.text == 'Найти игры':
                user.position = 'find_games'
                await bot.send_message(user.pid, 'Выберите, что вы хотите', reply_markup = find_games_markup)
            elif message.text == 'Найти песни':
                user.position = 'find_songs'
                await bot.send_message(user.pid, 'Выберите, что вы хотите', reply_markup = find_songs_markup)
            else:
                 await bot.send_message(user.pid, 'Неизвестная команда')
                    
        elif user.position == 'find_games':
            if message.text == 'Рандомная игра':
                avaliable_games = user.avaliable_games()
                random_index = random.randint(0, len(avaliable_games) - 1)
                random_game = avaliable_games[random_index]
                await bot.send_message(user.pid, random_game.get_description(), parse_mode = 'Markdown')
            elif message.text == 'Рандомная игра определенной категории':
                user.position = 'random_game_for_aim'
                await bot.send_message(user.pid, 'Выберите интересующую вас категорию', reply_markup=aims_markup)
            elif message.text == 'Найти игры по фильтрам':
                user.position = 'find_game_by_filter'
                user.filter_ans = []
                await bot.send_message(user.pid, 'Выберите категорию', reply_markup=aims_markup)
            elif message.text == 'Назад':
                user.position = 'main_menu'
                await bot.send_message(user.pid, 'Перехожу в главное меню', reply_markup = main_menu_markup)
            else:
                bot.send_message(user.pid, 'Неизвестная команда')
                
        elif user.position == 'find_game_by_filter':
            user.filter_ans.append(message.text)
            n = len(user.filter_ans) - 1
            if n == len(questions_filter):
                nes_aim = user.filter_ans[0]
                nes_activity = user.filter_ans[1]
                nes_age = user.filter_ans[2]
                nes_num_of_parts = user.filter_ans[3]
                nes_time = user.filter_ans[4]
                avaliable_games = user.avaliable_games()
                res = ''
                for game in avaliable_games:
                    if nes_aim == 'Неважно' or game.aim == nes_aim:
                        if nes_activity == 'Неважно' or game.activity == nes_activity:
                            if nes_age == 'Неважно' or nes_age in game.age:
                                if nes_num_of_parts == 'Неважно' or game.time == nes_num_of_parts:
                                    if nes_time == 'Неважно' or game.time == nes_time:
                                        res += game.get_description()
                if len(res) == 0:
                    res = 'Ой... Игр с заданными критериями не нашлось'
                await bot.send_message(user.pid, res, reply_markup = find_games_markup, parse_mode = 'Markdown', disable_web_page_preview = True)
                user.position = 'find_games'
            else:
                await bot.send_message(user.pid, questions_filter[n], reply_markup = markups_filter[n])
        
        elif user.position == 'random_game_for_aim':
            mess = ''
            avaliable_games = user.avaliable_games()
            need_games = []
            for game in avaliable_games:
                if game.aim == message.text or message.text == 'Неважно':
                    need_games.append(game)
            random_index = random.randint(0, len(need_games) - 1)
            random_game = need_games[random_index]
            await bot.send_message(user.pid, random_game.get_description(), parse_mode = 'Markdown', reply_markup = find_games_markup)
        
        elif user.position == 'find_songs':            
            if message.text == 'Рандомная песня':
                avaliable_songs = user.avaliable_songs()
                random_index = random.randint(0, len(avaliable_songs) - 1)
                random_song = avaliable_songs[random_index]
                await bot.send_message(user.pid, random_song.get_description(), parse_mode = 'Markdown')
            elif message.text == 'Все песни':
                avaliable_songs = user.avaliable_songs()
                message = ''
                for song in avaliable_songs:
                    message += song.get_description()
                    message += '\n'
                await bot.send_message(user.pid, message, parse_mode = 'Markdown', disable_web_page_preview = True)
            elif message.text == 'Назад':
                user.position = 'main_me\nu'
                await bot.send_message(user.pid, 'Перехожу в главное меню', reply_markup = main_menu_markup)
                user.position = 'main_menu'
                
                
        elif user.position == 'my_account':
            if message.text == 'Назад':
                await bot.send_message(user.pid, 'Перехожу в главное меню', reply_markup = main_menu_markup)
                user.position = 'main_menu'
            elif message.text == 'Изменить данные аккаунта':
                await bot.send_message(user.pid, 'Выберете, что вы хотите изменить', reply_markup = change_account_markup)
                user.position = 'change_account_data'
                
                
        elif user.position == 'change_account_data':
            if message.text == 'Назад':
                await bot.send_message(user.pid, 'Перехожу назад', reply_markup = my_account_markup)
                user.position = 'my_account'
            elif message.text == 'Имя':
                await bot.send_message(user.pid, 'Введите ваше новое имя', reply_markup = my_account_markup)
                user.position = 'change_name'
            elif message.text == 'Возраст':
                await bot.send_message(user.pid, 'Введите ваш новый возраст', reply_markup = my_account_markup)
                user.position = 'change_age'
            else:
                await bot.send_message(user.pid, 'Вы можете менять либо имя, либо возраст!')
        
        
        elif user.position == 'change_name':
            user.name = message.text
            await bot.send_message(user.pid, f'Приятно познакомиться снова, {user.name}')
            await sleep(0.5)
            await bot.send_message(user.pid, user.getinfo(), reply_markup = my_account_markup)
            user.position = 'my_account'
            #update_db(user) ИЗМЕНЕНИЕ БД
        
        
        elif user.position == 'change_age':
            user.age = message.text
            await bot.send_message(user.pid, f'Ваш возраст изменен на {user.age}')
            await sleep(0.5)
            await bot.send_message(user.pid, user.getinfo(), reply_markup = my_account_markup)
            user.position = 'my_account'
            #update_db(user) ИЗМЕНЕНИЕ БД

    
    
    
    
    
    elif user in current_register.keys():
        registration_answers[user].append(message.text)
        current_register[user] += 1
        if current_register[user] == len(registration_question):
            await bot.send_message(user, 'Отлично! Вы зарегистрировались. Передим в главное меню', reply_markup = main_menu_markup)
            current_register.pop(user)
            answers = registration_answers[user]
            registered_users[user] = User(answers[0], answers[1], user)
            #add_user_db(registered_users[user]) !! ОБНОВЛЕНИЕ БДШКИ
            return
        await bot.send_message(user, registration_question[current_register[user]])
    else:
        await bot.send_message(user, 'Прежде чем начать использовать бот, необходимо зарегистрироваться! Введите команду /register')


await bot.polling()

HERE!!!


In [536]:
registered_users[581791415].filter_ans

['Знакомство', 'Неважно', 'Средний', '10-30']